In [1]:
from database.market import Market
import pandas as pd
from tqdm import tqdm
from datetime import datetime,timezone
import pandas_datareader as pdr
from extractor.finnhub_extractor import FinnhubExtractor
from extractor.tiingo_extractor import TiingoExtractor
from time import sleep

In [2]:
start = "2008-01-01"
end = datetime.now().strftime("%Y-%m-%d")
market = Market()

In [9]:
# market.connect()
# market.drop_table("sp500")
# df = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies",attrs={"id":"constituents"},header=0)[0]
# market.store_data("sp500",df)
# market.close()

In [3]:
market.connect()
tickers = market.retrieve_data("sp500").sort_values("Symbol")["Symbol"]
market.close()

In [5]:
market.connect()
market.drop_table("new_pdr_prices")
market.drop_table("new_tiingo_prices")
market.drop_table("new_finnhub_prices")
fails = []
for ticker in tqdm(tickers):
    try:
        if "." in ticker:
            ticker = ticker.replace(".","-")
        status = "tiingo"
        price = pdr.DataReader(ticker, 
                       start=start, 
                       end=end, 
                       data_source='yahoo')
        price["ticker"] = ticker
        market.store_data("new_pdr_prices",price.reset_index())
        tp = TiingoExtractor(ticker,start,end).extract()
        tp["ticker"] = ticker
        market.store_data("new_tiingo_prices",tp.reset_index())
        fp = FinnhubExtractor.extract(ticker,start,end)
        fp = pd.DataFrame(fp)
        fp["ticker"] = ticker
        market.store_data("new_finnhub_prices",fp.reset_index())
        sleep(8)
    except Exception as e:
        msg = {"status":status,"ticker":ticker,"exception":str(e)}
        print(msg,ticker)
        fails.append(ticker)
market.close()

 25%|███████████████████████████████████████████▌                                                                                                                                      | 124/506 [20:58<47:11,  7.41s/it]

{'status': 'tiingo', 'ticker': 'CXO', 'exception': 'No data fetched for symbol CXO using YahooDailyReader'} CXO


 25%|███████████████████████████████████████████▉                                                                                                                                      | 125/506 [21:09<52:57,  8.34s/it]

{'status': 'tiingo', 'ticker': 'D', 'exception': "('Connection broken: IncompleteRead(0 bytes read)', IncompleteRead(0 bytes read))"} D


 86%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                        | 436/506 [1:15:04<09:05,  7.79s/it]

...

Expecting value: line 1 column 1 (char 0)
{'status': 'tiingo', 'ticker': 'ZBRA', 'exception': "'NoneType' object does not support item assignment"} ZBRA


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋| 505/506 [1:27:13<00:05,  5.74s/it]

Expecting value: line 1 column 1 (char 0)
{'status': 'tiingo', 'ticker': 'ZION', 'exception': "'NoneType' object does not support item assignment"} ZION


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 506/506 [1:27:14<00:00, 10.35s/it]

Expecting value: line 1 column 1 (char 0)
{'status': 'tiingo', 'ticker': 'ZTS', 'exception': "'NoneType' object does not support item assignment"} ZTS
